# Extraction d'Embeddings d'Images avec ResNet50 et Affichage d'Images Similaires

Ce notebook présente un processus complet pour :

- Charger un modèle pré-entraîné (ResNet50) et en extraire les embeddings des images.
- Construire le chemin complet des images en évitant la duplication des dossiers.
- Charger un DataFrame à partir d'un fichier CSV contenant des chemins d'images.
- Extraire les embeddings pour chaque image du DataFrame.
- Sélectionner aléatoirement quelques images et calculer les distances (similitude) entre les embeddings.
- Afficher en console et graphiquement les images similaires.

⚠️ **Note** : Les chemins indiqués dans ce notebook sont adaptés pour un environnement Windows. Adaptez-les si vous utilisez Colab (par exemple, en montant votre Google Drive).

In [ ]:
# Importation des bibliothèques nécessaires
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import random
import torch
from torchvision import models, transforms

# Pour afficher correctement les images dans le notebook
%matplotlib inline

## 1. Fonction pour Construire le Chemin Complet de l'Image

Cette fonction permet de construire le chemin complet vers une image en supprimant une éventuelle duplication du dossier de base. Par exemple, si le chemin relatif contient déjà le nom du dossier de base, il sera retiré.

In [ ]:
def build_image_path(base_path, relative_path):
    """
    Construit le chemin complet vers l'image en supprimant une éventuelle duplication
    du dossier de base (ex. 'Logos_dataset') si présent dans le chemin relatif.
    """
    base_folder = os.path.basename(os.path.normpath(base_path))
    if relative_path.startswith(base_folder):
        relative_path = relative_path[len(base_folder):].lstrip("/\\")
    return os.path.join(base_path, relative_path)

## 2. Configuration du Modèle et du Pipeline de Prétraitement

Nous utilisons le modèle pré-entraîné **ResNet50** de Torchvision. 
Le modèle est mis en mode évaluation et déplacé sur le device disponible (GPU si possible). 
Nous retirons la dernière couche fully connected afin d'obtenir les embeddings de l'image.

Le pipeline de prétraitement redimensionne l'image, effectue un crop centré, convertit l'image en tenseur et normalise les valeurs.

In [ ]:
# Définir le device global (GPU si disponible, sinon CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Chargement du modèle pré-entraîné ResNet50
model = models.resnet50(pretrained=True)
model.eval()  # Mode évaluation
model.to(device)  # Déplacer le modèle sur le device

# Retirer la dernière couche fully connected pour obtenir les embeddings
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.to(device)

# Définition du pipeline de prétraitement
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # Conversion en tensor avec valeurs dans [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## 3. Fonction pour Extraire l'Embedding d'une Image

La fonction `get_embedding` ouvre une image, applique le prétraitement, et passe l'image dans le modèle pour obtenir un vecteur d'embedding. 
Le vecteur est aplati pour obtenir une représentation 1D de l'image.

In [ ]:
def get_embedding(image_path):
    """
    Extrait l'embedding d'une image en utilisant le modèle ResNet50 pré-entraîné.
    
    Args:
        image_path (str): Chemin vers l'image.
    
    Returns:
        torch.Tensor: Vecteur d'embedding aplati.
    """
    try:
        img = Image.open(image_path).convert('RGB')
    except Exception as e:
        raise ValueError(f"Erreur lors de l'ouverture de l'image {image_path} : {e}")
    
    # Appliquer le prétraitement
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)  # Ajout de la dimension batch
    
    # Déplacer le batch sur le device
    input_batch = input_batch.to(device)
    
    with torch.no_grad():
        features = feature_extractor(input_batch)
    
    # Aplatir le résultat pour obtenir un vecteur 1D
    features = features.view(features.size(0), -1)
    return features

## 4. Chargement des Données et Extraction des Embeddings

Dans cette section, nous :

- Chargeons un DataFrame à partir d'un fichier CSV contenant une colonne `main_mark_image`.
- Définissons le chemin de base où se trouvent les images.
- Pour chaque image, nous utilisons la fonction `build_image_path` pour construire le chemin complet et la fonction `get_embedding` pour extraire l'embedding.

Les embeddings sont stockés dans un dictionnaire associant le chemin relatif de l'image à son embedding.

In [ ]:
# Chemin vers le fichier CSV contenant la colonne 'main_mark_image'
csv_path = r"C:\Users\HP\Desktop\LIPSTIP\extracted_paths_final_LIPSTIP.csv"  # Mettez à jour ce chemin si nécessaire

# Chargement du DataFrame
df = pd.read_csv(csv_path)

# Chemin de base où se trouvent les images
base_path = r"C:\Users\HP\Desktop\LIPSTIP\Logos_dataset"

# Extraction des embeddings pour chaque image
embeddings = {}
for idx, row in df.iterrows():
    # Construction du chemin complet de l'image
    main_img_path = build_image_path(base_path, row['main_mark_image'])
    # Extraction de l'embedding et déplacement sur CPU
    emb = get_embedding(main_img_path).cpu().numpy().flatten()
    embeddings[row['main_mark_image']] = emb

## 5. Sélection d'Images Aléatoires et Affichage des Images Similaires

Nous sélectionnons 3 images aléatoires à partir des embeddings extraits. 
Pour chaque image sélectionnée, nous calculons la distance euclidienne avec toutes les autres images pour identifier les 5 images les plus proches. 

Les résultats sont affichés dans la console.

In [ ]:
# Sélection aléatoire de 3 images parmi celles extraites
selected_images = random.sample(list(embeddings.keys()), 3)

# Affichage en console des 5 images les plus proches pour chaque image sélectionnée
for img in selected_images:
    distances = {}
    for other_img, emb in embeddings.items():
        if other_img == img:
            continue
        dist = np.linalg.norm(embeddings[img] - emb)
        distances[other_img] = dist
    similar_images = sorted(distances, key=distances.get)[:5]
    print(f"\nPour l'image {img}, les 5 plus proches sont :")
    for sim_img in similar_images:
        print(sim_img)

## 6. Fonction d'Affichage Graphique des Images Similaires

Cette fonction affiche l'image de référence et les images similaires sur une même figure. 
Chaque image est chargée via son chemin complet, puis affichée avec un titre approprié.

In [ ]:
def display_similar_images(reference_image, similar_images):
    plt.figure(figsize=(15, 5))
    
    # Affichage de l'image de référence
    plt.subplot(1, len(similar_images) + 1, 1)
    ref_img = Image.open(build_image_path(base_path, reference_image))
    plt.imshow(ref_img)
    plt.title("Référence")
    plt.axis('off')
    
    # Affichage des images similaires
    for i, sim in enumerate(similar_images):
        plt.subplot(1, len(similar_images) + 1, i + 2)
        sim_img = Image.open(build_image_path(base_path, sim))
        plt.imshow(sim_img)
        plt.title(f"Sim {i+1}")
        plt.axis('off')
    plt.show()

## 7. Affichage Graphique des Images Similaires

Pour chaque image sélectionnée, nous recalculons les distances et affichons graphiquement l'image de référence et ses 5 images les plus similaires.

In [ ]:
for img in selected_images:
    distances = {}
    for other_img, emb in embeddings.items():
        if other_img == img:
            continue
        dist = np.linalg.norm(embeddings[img] - emb)
        distances[other_img] = dist
    similar_images = sorted(distances, key=distances.get)[:5]
    display_similar_images(img, similar_images)